# Bosonic Double-well

In [ ]:
include("../src/KadanoffBaym.jl")
using OrdinaryDiffEq # for ODEProblem and solve
using RecursiveArrayTools # For ArrayPartition
using PyPlot
using PyCall
qt = pyimport("qutip")
np = pyimport("numpy")

PyPlot.matplotlib.rc("text", usetex=true)
PyPlot.matplotlib.rc("font", family="serif", size=16)

## Model

### Non-Hermitian Hamiltonian:

$$
    \hat{H} = \left(\omega_1 -i\lambda/2\right) a^{\dagger}_1 a^\phantom{\dagger}_1 + \omega_2 a^{\dagger}_2 a^\phantom{\dagger}_2 + J \left(a^{\dagger}_1 a^\phantom{\dagger}_2 + a^{\dagger}_2 a^\phantom{\dagger}_1\right)
$$

### Master equation:

$$
    \partial_{t} \hat{\rho}=-i\left[\hat{H} \hat{\rho}-\hat{\rho} \hat{H}^{
    \dagger}\right]+\lambda a^\phantom{\dagger}_1 \hat{\rho}  a^{\dagger}_1
$$

### Equations of motion

#### Vertical Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    i \partial_t - \omega_1 + i\lambda/2 & -J \\
    -J & i \partial_t - \omega_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t')    \\
    0 &= \begin{pmatrix}
    i \partial_t - \omega_1 - i\lambda/2 & -J \\
    -J & i \partial_t - \omega_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t')  
    + i\lambda
    \begin{pmatrix}
    G^T_{11} & G^T_{12} \\
    0 & 0
    \end{pmatrix}(t, t')   
\end{split}\end{align}

#### Horizontal Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \omega_1 + i\lambda/2 & J \\
    J & i \partial_{t'} + \omega_2
    \end{pmatrix} \\
    0 &= \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \omega_1 - i\lambda/2 & J \\
    J & i \partial_{t'} + \omega_2
    \end{pmatrix}  
    + i\lambda
    \begin{pmatrix}
    G^{\tilde{T}}_{11} & 0 \\
    G^{\tilde{T}}_{21} & 0
    \end{pmatrix}(t, t')   
\end{split}\end{align}

#### Equal-Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    i \partial_T + i\lambda & 0 \\
    0 & i \partial_T
    \end{pmatrix} 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)
    - \left[\begin{pmatrix}
    0 & J \\
    J & 0
    \end{pmatrix}, 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)\right] \\
    0 &= \begin{pmatrix}
    i \partial_T - i\lambda & 0 \\
    0 & i \partial_T
    \end{pmatrix} 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)  
    - \left[\begin{pmatrix}
    0 & J \\
    J & 0
    \end{pmatrix}, 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)\right]
    + i\lambda
    \begin{pmatrix}
    G^T_{11} + G^{\tilde{T}}_{11} & G^T_{12} \\
    G^{\tilde{T}}_{21} & 0
    \end{pmatrix}(T, 0)   
\end{split}\end{align}

## Solving

In [ ]:
# time parameters
T = 1.0
dt = 2^-8 # this algorithm requires a fixed time
n = Int(1/dt) 

# Non-Hermitian Hamiltonian and jump operator
ω₁ = 0.0
ω₂ = 0.0
J = 2*pi*1.0/2
λ = 1.0
h = ComplexF64[ω₁ - 0.5im * λ J; J ω₂]

In [ ]:
# quantum numbers
dim = 2

# Define your Green functions at (t0, t0), time-arguments at the beginning
Lesser0 = KadanoffBaym.LesserGF(zeros(ComplexF64, 1, 1, dim, dim))
Greater0 = KadanoffBaym.GreaterGF(zeros(ComplexF64, 1, 1, dim, dim))

# initial condition
N_0 = 1

Lesser0[1,1,1,1] = -1.0im * N_0
Greater0[1,1,1,1] = -1.0im * (N_0 + 1)
Greater0[1,1,2,2] = -1.0im

# Pack them in an ArrayPartition
u0 = ArrayPartition(Lesser0, Greater0);

# Remember that `u` here is also an ArrayPartition-like element
function f(u, p, t, t′)
    u1, u2 = u.x[1], u.x[2]
    rhs1 = -1.0im * h * u1[t,t′]
    rhs2 = -1.0im * (h * u2[t,t′] - [[1.0im * λ, 0] [0, 0]] * u2[t,t′])
  return ArrayPartition(rhs1, rhs2)
end

function f_diag(u, p, t, t′)
    u1, u2 = u.x[1], u.x[2]
    rhs1 = -1.0im * (h * u1[t,t′] - u1[t,t′] * adjoint(h))
    rhs2 = -1.0im * (adjoint(h) * u2[t,t′] - u2[t,t′] * h
        - 1.0im * λ * [[u1[t,t′, 1, 1] + u2[t,t′, 1, 1], u2[t,t′, 2, 1]] [u2[t,t′, 1, 2], 0]])
  return ArrayPartition(rhs1, rhs2)
end

# ODE problem is defined by the rhs, initial condition and time span
prob = ODEProblem(f, u0, (0.0, T), f_diag=nothing)

# Algorithm to timestep is the (Kadanoff-Baym) ABM43 (only really this one exists)
alg = KadanoffBaym.KB{ABM43}()

sol = solve(prob, alg, dt);

In [ ]:
stop = Int(n/2) + 1

times = range(0, length=n + 1, stop=T) |> collect;

# defining the tau times
times_tau = 2 .* vcat([-times[stop - (k - 1)] for k in 1:stop - 1], times[1:stop]);

## QuTiP benchmark

In [ ]:
# maximal number of photons
# Fock-space dimension will be n_max + 1
n_max = 3

# initial state
psi0_list = [qt.basis(n_max + 1, N_0), qt.basis(n_max + 1, 0)]
psi0 = qt.tensor(psi0_list)

# define annihilation operators
a_1_list = [qt.destroy(n_max + 1), qt.qeye(n_max + 1)]
a_1  = qt.tensor(a_1_list)    
a_2_list = [qt.qeye(n_max + 1), qt.destroy(n_max + 1)]
a_2  = qt.tensor(a_2_list)    

# Hamiltonian
H  = J * a_1.dag() * a_2
H += H.dag();
H += ω₁ * a_1.dag() * a_1 + ω₂ * a_2.dag() * a_2; # do not give qutip a non-Hermitian Hamiltonian

observables = [a_1.dag()*a_1, a_2.dag()*a_2];

In [ ]:
j_ops = [sqrt(λ) * a_1] # qutip only needs the jump operator

# quickly solve once for observables
me = qt.mesolve(H, psi0, times, j_ops, observables)

# solve for the time-dependent density matrix
t_sols = qt.mesolve(H, psi0, times, j_ops); # t_sols.states returns density matrices

#### Two times

Calculate $\langle a^\dagger_1(t') a^\phantom{\dagger}_1(t)\rangle$, where $t' = t + \tau$.

In [ ]:
tau_t_sols = Dict()
for k in 1:length(t_sols.states)
    tau_t_sols[k] = qt.mesolve(H, a_1 * t_sols.states[k], times, j_ops).states
end

a_1_dag_a_1 = zeros(ComplexF64, length(t_sols.states), length(t_sols.states))
for k in 1:length(t_sols.states)
    for l in 1:length(t_sols.states)
        a_1_dag_a_1[k, l] = (a_1.dag() * tau_t_sols[k][l]).tr()    
    end
end

In [ ]:
# reshape the above array to fit into our two-time "matrix" structure 
# see the plot below for illustration
unskewed_a_1_dag_a_1 = zeros(ComplexF64, length(t_sols.states), 2*length(t_sols.states) - 1)
for (k, x) in enumerate([a_1_dag_a_1[k, :] for k in 1:length(t_sols.states)])
    for (l, y) in enumerate(x)
        ind = k + l - 1 # verify the -1 relative to the original python code
        unskewed_a_1_dag_a_1[k, ind] = y  
    end
end

In [ ]:
figure(figsize=(6, 2))
subplot(121)
imshow(real(a_1_dag_a_1), cmap="plasma")

subplot(122)
imshow(real(unskewed_a_1_dag_a_1), cmap="plasma")

tight_layout()

## Plotting

In [ ]:
# time diagonal
GL_11 = [-imag(sol.u.x[1][k, k, 1, 1]) for k in 1:n+1]
GL_22 = [-imag(sol.u.x[1][k, k, 2, 2]) for k in 1:n+1];

In [ ]:
# building the tau Green functions
GL_11_tau = [sol.u.x[1][stop - (k - 1), stop + (k - 1), 1, 1] for k in 1:stop]
GL_11_tau_reversed = [GL_11_tau[stop - (k - 1)] for k in 1:stop];
GL_11_tau_full = vcat(-conj(GL_11_tau_reversed[1:end-1]), GL_11_tau);

In [ ]:
a_1_dag_a_1_tau = [unskewed_a_1_dag_a_1[stop - (k - 1), stop + (k - 1)] for k in 1:stop]
a_1_dag_a_1_tau_reversed = [a_1_dag_a_1_tau[stop - (k - 1)] for k in 1:stop]
a_1_dag_a_1_tau_full = vcat(conj(a_1_dag_a_1_tau_reversed[1:end-1]), a_1_dag_a_1_tau);

In [ ]:
figure(figsize=(12, 3))
subplot(121)
plot(times, GL_11)
plot(times, me.expect[1], c="k", ls="--")
plot(times, [real(unskewed_a_1_dag_a_1[k, k]) for k in 1:n+1], c="r", ls=":", lw=3.0, alpha=0.5)
plot(times, GL_22)
plot(times, me.expect[2], c="k", ls="--")
xlim(0, T)
ylim(0, N_0)
xlabel("\$t\$")

subplot(122)
plot(times_tau, -imag(GL_11_tau_full))
plot(times_tau, a_1_dag_a_1_tau_full, c="k", ls="--")
xlim(-T, T)
xlabel("\$\\tau\$")
# ylim(0, 1.0)

tight_layout()
# savefig("test.pdf")

In [ ]:
figure(figsize=(12, 3))
subplot(121)
plot(times, GL_11 - me.expect[1], c="k", ls="--")
plot(times, GL_22 - me.expect[2], c="k", ls="--")
xlim(0, T)
# ylim(0, N_0)
xlabel("\$t\$")

subplot(122)
plot(times_tau, -imag(GL_11_tau_full) - a_1_dag_a_1_tau_full, c="k", ls="--")
xlim(-T, T)
xlabel("\$\\tau\$")
# ylim(0, 1.0)

tight_layout()